# DEA Notebook 

![alt text](../dea-gallery-crop.png "DEA")

### DEA is a Content creation service and no-code platform for DestinE storytelling and data visualization

## From data to story

This notebook contains steps for:
- Generating a Cloud Optimized GeoTIFF (COG) file from data provided by the DestinE CacheB service (ERA5 t2m temperature monthly mean over Europe)
- Uploading the  COG file as a user asset in DEA
- Creating a story with a slide containing the asset

# Install pre-requirements

In [6]:
!pip install matplotlib
!pip install numpy
!pip install xarray
!pip install rasterio
!pip install requests
!pip install rio-cogeo
!pip install rioxarray
!pip install zarr 
!pip install fsspec
!pip install aiohttp 
!pip install dask

# Download the sample NetCDF file and save it in the input folder

DestinE Platform Service CacheB will be used to download an ERA5 t2m data selecting 2025-03 as date and Europe extent as area.

The data will be saved locally in NetCDF format.

**NOTE**: the temporary password is valid for a limited period of time and needs to be regenerated and reconfigured periodically by running the cells above.

The following file will be downlaoded locally: **t2m_era5_europe_20250321.nc**

## Insert DestinE Platform credentials

In [32]:
%%capture cap
%run ./../../cacheb/cacheb-authentication.py

Username:  cristinaarcari@alia-space.com
Password:  ········


In [33]:
output_1 = cap.stdout.split('}\n')
token = output_1[-1][0:-1]
from pathlib import Path
with open(Path.home() / ".netrc", "w") as fp:
    fp.write(token)

## Define variables

In [34]:
file_path = "t2m_europe_era5_202503.nc"
tif_path = "t2m_europe_era5_202503.tiff"

## Get data

Download data from DestinE Platform CacheB

In [36]:
import xarray as xr


data = xr.open_dataset(
        "https://cacheb.dcms.destine.eu/era5/reanalysis-era5-single-levels-monthly-means-v0.zarr",
        engine="zarr",
        storage_options={"client_kwargs": {"trust_env": "true"}},
        chunks={},
    )

# Convert to Celsius

t2m = data.t2m.astype("float32") - 273.15
t2m.attrs["units"] = "C"

t2m = t2m.assign_coords(longitude=(t2m['longitude'] % 360))  # First ensure longitude is in 0-360 range
t2m['longitude'] = xr.where(t2m['longitude'] > 180, t2m['longitude'] - 360, t2m['longitude'])  # Then shift to -180 to 180

# Sort by longitude to maintain the proper order
t2m = t2m.sortby('longitude')

t2m = t2m.assign_coords(latitude=(t2m['latitude'] % 180))  # First ensure latitude is in 0-180 range
t2m['latitude'] = xr.where(t2m['latitude'] > 90, t2m['latitude'] -180, t2m['latitude'])  # Then shift to -90 to 90

# Sort by latitude to maintain the proper order
t2m = t2m.sortby('latitude')

# Select an area
t2m_europe_area = t2m.sel(**{"latitude": slice(34.50, 81.01), "longitude": slice(-31.27, 69.06)})

print(f"Data to download...")
print(t2m_europe_area)

#t2m_europe_area = t2m_europe_area.compute()

# get a single time step
t2m_slice = t2m_europe_area.sel(valid_time="2025-03")

print(t2m_slice)

t2m_slice = t2m_slice.compute()

# save as NetCDF
t2m_slice.to_netcdf(file_path)

print(f"Your file {file_path} is ready!")


Data to download...
<xarray.DataArray 't2m' (valid_time: 1024, latitude: 187, longitude: 402)> Size: 308MB
dask.array<getitem, shape=(1024, 187, 402), dtype=float32, chunksize=(120, 187, 256), chunktype=numpy.ndarray>
Coordinates:
    number      int64 8B 0
    surface     float64 8B 0.0
  * valid_time  (valid_time) datetime64[ns] 8kB 1940-01-01 ... 2025-04-01
  * longitude   (longitude) float64 3kB -31.25 -31.0 -30.75 ... 68.5 68.75 69.0
  * latitude    (latitude) float64 1kB 34.5 34.75 35.0 35.25 ... 80.5 80.75 81.0
Attributes:
    units:    C
<xarray.DataArray 't2m' (valid_time: 1, latitude: 187, longitude: 402)> Size: 301kB
dask.array<getitem, shape=(1, 187, 402), dtype=float32, chunksize=(1, 187, 256), chunktype=numpy.ndarray>
Coordinates:
    number      int64 8B 0
    surface     float64 8B 0.0
  * valid_time  (valid_time) datetime64[ns] 8B 2025-03-01
  * longitude   (longitude) float64 3kB -31.25 -31.0 -30.75 ... 68.5 68.75 69.0
  * latitude    (latitude) float64 1kB 34.5 34.75

## Wait until your NetCDF file appear in your workspace before executing the next cell. 

>**_NOTE:_** The download of the file should take about 1 minute. The file _t2m_europe_era5_202503.nc_ should appear in the notebook workspace

# Convert NetCDF to GeoTiff

The following file will be downlaoded locally: **t2m_europe_era5_202503.tiff**

In [37]:
import xarray as xr
import rioxarray

# Open dataset and select variable (e.g. var='sst')

var = 't2m'
data = xr.open_dataset(file_path)
data = data[var]

# Apply CRS (assuming the coordinates are longitude and latitude)

var = data.rename({'latitude': 'y', 'longitude': 'x'})
var.rio.write_crs("EPSG:4326", inplace=True)


<xarray.DataArray 't2m' (valid_time: 1, y: 187, x: 402)> Size: 301kB
[75174 values with dtype=float32]
Coordinates:
    number       int64 8B ...
    surface      float64 8B ...
  * valid_time   (valid_time) datetime64[ns] 8B 2025-03-01
  * x            (x) float64 3kB -31.25 -31.0 -30.75 -30.5 ... 68.5 68.75 69.0
  * y            (y) float64 1kB 34.5 34.75 35.0 35.25 ... 80.25 80.5 80.75 81.0
    spatial_ref  int64 8B 0
Attributes:
    units:    C

## Manage NaN values

If data contains NaN, replace it with -9999 to make them transparent afterwards (as soon as you then map this value to black in the colormap text file)


In [38]:
var_filled = var.fillna(-9999)

## Rotate data in case of need 

it can be necessary to flip data of 180 degrees; verify if this step is necessary to your case

```
var_filled = var_filled[::-1, ::]
```

## Save the transformed data to GeoTiff (intermediate step)

Wait untill the tiff file is created (it could take a while)

In [40]:
var_filled.rio.to_raster(tif_path)

print(f"Your GeoTiff file in grayscale {tif_path} is ready!")

Your GeoTiff file in grayscale t2m_europe_era5_202503.tiff is ready!


# Apply the colormap to the generated Tiff in grayscale

Matplotlib will be used to achieve this.

The following file will be downlaoded locally: **t2m_era5_202503_rgb.tiff**

## 1. Load the colormap from the file

The file colormap.txt contains a colormap suitable for temperature in Celsius.

In [41]:
import rasterio
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

# Load the colormap from the file
def load_colormap(colormap_file):
    colormap = []
    values = []
    with open(colormap_file, 'r') as f:
        for line in f:
            # Skip empty lines
            if not line.strip():
                continue
            parts = line.strip().split(maxsplit=1)  # Split only at the first space
            value = float(parts[0])
            # Ensure correct parsing of the RGB values
            rgb = tuple(map(int, parts[1].replace(',', ' ').split()))
            values.append(value)
            colormap.append(rgb)
    return values, colormap

values, colormap = load_colormap('colormap.txt')

## 2. Create a custom colormap for matplotlib

In [42]:
def create_colormap(values, colormap):
    norm_values = np.linspace(0, 1, len(values))
    colors = np.array(colormap) / 255.0  # Normalize RGB to [0, 1] range
    return ListedColormap(colors), norm_values

cmap, norm_values = create_colormap(values, colormap)

## 3. Open the grayscale TIFF

In [43]:
with rasterio.open(tif_path) as src:
    grayscale = src.read(1)  # Read the first band

## 4. Apply the colormap

Interpolate the grayscale values to teh colormap

A new Tiff will be generated

In [44]:
output_rgb_tiff = "t2m_europe_era5_202503_rgb.tiff"

In [46]:
def apply_colormap_to_tiff(input_tiff, output_tiff, values, colormap):
    with rasterio.open(input_tiff) as src:
        grayscale = src.read(1)  # Read the first band (grayscale)
        
        # Apply the colormap to the grayscale data
        normalized = (grayscale - np.min(grayscale)) / (np.max(grayscale) - np.min(grayscale))  # Normalize
        cmap = ListedColormap(np.array(colormap) / 255.0)  # Convert colormap to matplotlib format
        rgb_image = cmap(normalized)

        # Write the RGB image to a new TIFF
        height, width, _ = rgb_image.shape
        profile = src.profile
        profile.update(count=3, dtype=rasterio.uint8)  # Update profile to handle RGB output

        with rasterio.open(output_tiff, 'w', **profile) as dst:
            dst.write((rgb_image[:, :, 0] * 255).astype(np.uint8), 1)
            dst.write((rgb_image[:, :, 1] * 255).astype(np.uint8), 2)
            dst.write((rgb_image[:, :, 2] * 255).astype(np.uint8), 3)

# Apply the colormap (assuming 'values' and 'colormap' were already loaded)
apply_colormap_to_tiff(tif_path, output_rgb_tiff, values, colormap)

print(f"Your file RGB {output_rgb_tiff} is ready!")

Your file RGB t2m_europe_era5_202503_rgb.tiff is ready!


# Convert TIFF to COG optimized for web

The following file will be downlaoded locally: **sst_era5_20250321_web_optimized.tiff**

First of all, import the needed dependencies

In [47]:
import rasterio
from rasterio.enums import Resampling
from rio_cogeo.cogeo import cog_validate, cog_translate
from rio_cogeo.profiles import cog_profiles
from datetime import datetime


## 1. Define paths to your files

In [48]:
now = datetime.now() # current date and time
current_time = now.strftime("%m%d%Y%H%M%S")

output_cog = "t2m_europe_era5_202503_cog.tiff"
output_cog_web_optimized = "t2m_europe_era5_202503_web_optimized_" + current_time + ".tiff"

## 2. Convert the GeoTiff to COG

In [49]:
cog_profile = cog_profiles.get("deflate")  # You can choose other profiles such as "jpeg", "lzw"

config = {
    "blocksize": 512,  # You can customize the blocksize
    "resampling": "average"  # Pass the resampling method as a string
}

with rasterio.open(output_rgb_tiff) as src:
    # Create COG
    cog_translate(
        src,
        output_cog,
        cog_profile,
        config=config,
        nodata=src.nodata,
        overview_level=8,  # Define the overview level
        overview_resampling="average",  # Pass the resampling method as a string
    )

print(f"COG {output_cog} created successfully!")

Reading input: <open DatasetReader name='t2m_europe_era5_202503_rgb.tiff' mode='r'>

Adding overviews...
Updating dataset tags...
Writing output to: t2m_europe_era5_202503_cog.tiff


COG t2m_europe_era5_202503_cog.tiff created successfully!


## 3. Convert COG to web-optimized COG using subprocess

In [50]:
import subprocess

subprocess.run([
    "rio", "cogeo", "create", 
    output_cog, 
    output_cog_web_optimized,
    "--blocksize", "512",
    "--overview-resampling", "average",
    "--overview-level", "8",
    "--web-optimized"
], check=True)

print(f"Web-optimized COG {output_cog_web_optimized} created successfully!")


Defining overview's `blocksize` to match the high resolution `blocksize`: 512


Reading input: /home/jovyan/desp-lab/dea-test/data2story/t2m_europe_era5_202503_cog.tiff

Adding overviews...
Updating dataset tags...
Writing output to: /home/jovyan/desp-lab/dea-test/data2story/t2m_europe_era5_202503_web_optimized_06052025222727.tiff


Web-optimized COG t2m_europe_era5_202503_web_optimized_06052025222727.tiff created successfully!


**NOTE**: The global COG file may result in missing the data over the poles, as those regions introduce distortions in the visualization.

# Upload the Web-Optimized COG on DEA!

You can now upload the COG file generated on DEA and use it in your stories using the DEA API.

> **_NOTE:_** You have to get a new access token to use the authenticated endpoint of DEA


In [51]:
%%capture cap
%run ./../dea-authentication.py

Type your username:  dea@alia-space.com
Type your password:  ········


In [52]:
output_2 = cap.stdout.split('}\n')
token_dea = output_2[-1][0:-1]

## Add the generated COG file to your DEA asset using DEA API

This file will be stored on your private workspace hosted on the DestinE Platform

> **_NOTE:_** As an alternative, you can upload your asset from DEA Story Editor

![alt text](./user_assets.png "Upload assets")

In [53]:
dea_base_url = "http://dea.destine.eu/"

### Import section

In [54]:
import json
import requests
import urllib.parse
import webbrowser

In [55]:


api_url = "api/userAssets/upload"
joined_url = urllib.parse.urljoin(dea_base_url, api_url)

payload = {'type': 'image/tiff',
'assetType': 'GeoTIFF/COG'}
files=[
  ('file',(output_cog_web_optimized,open(output_cog_web_optimized,'rb'),'image/tiff'))
]
headers = {
  'Authorization': f'Bearer {token_dea}'
}

response = requests.request("POST", joined_url, headers=headers, data=payload, files=files)
userid = ''
numbytes = ''
assetId = ''
response_data = None

if response.status_code == 201:
    print('Asset uploaded succesfully!')
    response_data = response.json()
    userid = response_data['asset']['userId']
    numbytes = response_data['asset']['bytes']
    assetId = response_data['asset']['_id']
else:
    print('Failed to upload asset')
    print(response.status_code)
    print(response.reason)
    



Asset uploaded succesfully!


## Create a story with your asset using DEA API

> **_NOTE:_** As an alternative, you can create your story from the [DEA Story Editor](https://dea.destine.eu/web/stories/editor)


### Type a title for your story

In [56]:
story_title = input("Type a title: ")

Type a title:  my data on europe


### Type a description for your story

In [58]:
story_description = input("Type a description: ")

Type a description:  era5 2m temperature in march 2025


In [59]:


story_url = "api/userStories"
new_url = urllib.parse.urljoin(dea_base_url, story_url)
story_editor_url = urllib.parse.urljoin(dea_base_url, 'web/stories')

# load the sample json template for API request body 
with open('data.json', 'r') as file:
    payload = json.load(file)

if story_title is not None:
    payload['title'] = story_title

if story_description is not None:
    payload['description'] = story_description
    
# replace tags in the JSON file with the proper asset's metadata
payload_string = json.dumps(payload)
payload_string = payload_string.replace('"<bytes>"', f'{numbytes}')
payload_string = payload_string.replace('<userid>', userid)
payload_string = payload_string.replace('<my_asset>', output_cog_web_optimized)
payload_string = payload_string.replace('<assetId>', assetId)

payload = json.dumps(json.loads(payload_string))




headers = {
  'Content-Type': 'application/json',
  'Authorization': f'Bearer {token_dea}'
}

res = requests.request("POST", new_url, headers=headers, data=payload)
res_data = None
storyId = '' 
story_editor_id_url = f'{story_editor_url}/editor/'

if res.status_code == 201:
    print('Story created succesfully!')
    res_data = res.json()
    storyId = res_data['_id']
    story_editor_id_url = f'{story_editor_url}/editor/{storyId}'
    print(f'You can edit your story at the following URL: {story_editor_id_url}')

else:
    print('Failed to create story') 
    print(res.status_code)
    print(res.reason)


Story created succesfully!
You can edit your story at the following URL: http://dea.destine.eu/web/stories/editor/68421a5bb62b202b54f093cc


## Your story has been created!

You can now continue to write your story from the [DEA Story Editor](https://dea.destine.eu/web/stories/editor).

Your story should look like the picture below:

 ![alt text](./new_story.png "Your new story")

You need to authenticate on the DestinE Platform to edit your story.

 ![alt text](./cover.png "DEA")